<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-pre-processing" data-toc-modified-id="Data-pre-processing-1">Data pre-processing</a></span></li><li><span><a href="#Gradient-Boosted-Trees" data-toc-modified-id="Gradient-Boosted-Trees-2">Gradient Boosted Trees</a></span><ul class="toc-item"><li><span><a href="#HistGradientBoostingClassifier" data-toc-modified-id="HistGradientBoostingClassifier-2.1">HistGradientBoostingClassifier</a></span><ul class="toc-item"><li><span><a href="#Initial-Classifiers" data-toc-modified-id="Initial-Classifiers-2.1.1">Initial Classifiers</a></span></li><li><span><a href="#Hyperparameter-tuning" data-toc-modified-id="Hyperparameter-tuning-2.1.2">Hyperparameter tuning</a></span></li><li><span><a href="#Including-the-V***-columns" data-toc-modified-id="Including-the-V***-columns-2.1.3">Including the V*** columns</a></span></li><li><span><a href="#Only-features-deemed-important-by-RF-model-interpretation" data-toc-modified-id="Only-features-deemed-important-by-RF-model-interpretation-2.1.4">Only features deemed important by RF model interpretation</a></span></li><li><span><a href="#Only-features-deemed-important-by-RF-model-interpretation-with-V***-features" data-toc-modified-id="Only-features-deemed-important-by-RF-model-interpretation-with-V***-features-2.1.5">Only features deemed important by RF model interpretation with V*** features</a></span></li></ul></li></ul></li></ul></div>

# Data pre-processing

In [2]:
from fastai.tabular.all import *
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer

from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.ensemble import HistGradientBoostingClassifier

path = Path('/Users/baranserajelahi/Codes/fraud-detection-pytorch-scikit-fastai/data')
Path.BASE_PATH = path
pd.options.display.max_rows = 200
pd.options.display.max_columns = 125

In [47]:
# Tabular object created and saved on the 02_ notebook
to = (path/'to_ss.pkl').load()

In [8]:
to["card1"].dtype

dtype('int16')

In [9]:
cats = ['ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 
       'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 
       'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'DeviceType', 'DeviceInfo', 'id_12', 
       'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 
       'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 
       'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36',
       'id_37', 'id_38']

for cat in cats:
    to[cat] = to[cat].astype('category')

In [10]:
to["card1"].dtype

CategoricalDtype(categories=[   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
                  ...
                  5634, 5635, 5636, 5637, 5638, 5639, 5640, 5641, 5642, 5643],
                 ordered=False)

In [60]:
xs, y = to.train.xs, to.train.y
valid_xs, valid_y = to.valid.xs, to.valid.y

# Gradient Boosted Trees

## HistGradientBoostingClassifier

In [112]:
'Previously, leaf-wise growth was an exclusive feature of lightGBM, but xgboost has since implemented this growth strategy (this change has not been reflected in the lightGBM docs, but has been acknowledged in a blog post). This strategy is only available for the histogram-based method (which I will explain below), so in order to use it, users will have to set the tree_method parameter to hist and the grow_policy parameter to lossguide.'

'Previously, leaf-wise growth was an exclusive feature of lightGBM, but xgboost has since implemented this growth strategy (this change has not been reflected in the lightGBM docs, but has been acknowledged in a blog post). This strategy is only available for the histogram-based method (which I will explain below), so in order to use it, users will have to set the tree_method parameter to hist and the grow_policy parameter to lossguide.'

In [113]:
'Xgboost offers the option tree_method=approx, which computes a new set of bins at each split using the gradient statistics. LightGBM and xgboost with the tree_method set to hist will both compute the bins at the beginning of training and reuse the same bins throughout the entire training process.'

'Xgboost offers the option tree_method=approx, which computes a new set of bins at each split using the gradient statistics. LightGBM and xgboost with the tree_method set to hist will both compute the bins at the beginning of training and reuse the same bins throughout the entire training process.'

In [114]:
'BOTH: Though lightGBM does not enable ignoring zero values by default, it has an option called zero_as_missing which, if set to True, will regard all zero values as missing. According to this thread on GitHub, lightGBM will treat missing values in the same way as xgboost as long as the parameter use_missing is set to True (which is the default behavior).'

'Though lightGBM does not enable ignoring zero values by default, it has an option called zero_as_missing which, if set to True, will regard all zero values as missing. According to this thread on GitHub, lightGBM will treat missing values in the same way as xgboost as long as the parameter use_missing is set to True (which is the default behavior).'

In [115]:
'LGBM: This is a method introduced in lightGBM that also takes advantage of the sparsity of large datasets. The essential observation behind this method is that the sparsity of features means that some features are never non-zero together. For instance, the words "Python" and "protein" might never appear in the same document in the data. This means that these features can be "bundled" into a single feature without losing any information. Suppose the tf-idf score for "Python" ranges from 0 to 10 and the tf-idf score for  "protein" ranges from 0 to 20. In this case, the feature'

'This is a method introduced in lightGBM that also takes advantage of the sparsity of large datasets. The essential observation behind this method is that the sparsity of features means that some features are never non-zero together. For instance, the words "Python" and "protein" might never appear in the same document in the data. This means that these features can be "bundled" into a single feature without losing any information. Suppose the tf-idf score for "Python" ranges from 0 to 10 and the tf-idf score for  "protein" ranges from 0 to 20. In this case, the feature'

In [116]:
'LGBM: Unfortunately, the problem of finding the most efficient bundle is NP-hard. Therefore, the authors of the paper opted for an approximate algorithm that tolerated a certain degree of overlap \gamma  between the non-zero elements within a feature bundle. The details of this algorithm are beyond the scope of this post, so please refer to the original paper for details.'

'Unfortunately, the problem of finding the most efficient bundle is NP-hard. Therefore, the authors of the paper opted for an approximate algorithm that tolerated a certain degree of overlap \\gamma  between the non-zero elements within a feature bundle. The details of this algorithm are beyond the scope of this post, so please refer to the original paper for details.'

HistGradientBoostingClassifier is based on Microsofts LightGBM.

### Initial Classifiers 

In [3]:
def m_roc(y, xs):
    preds = clf.predict_proba(xs)
    preds_isFraud = preds[:,1]
    return roc_auc_score(y, preds_isFraud)

The max_iter parameter is increased to 1000 from its default value of 100.

In [22]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1,
                                    max_iter=1000, scoring='roc_auc')
clf.fit(xs,y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

Binning 0.042 GB of training data: 0.286 s
Binning 0.005 GB of validation data: 0.021 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.83480, val score: 0.78900, in 0.062s
[2/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.86395, val score: 0.81880, in 0.073s
[3/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.86796, val score: 0.81789, in 0.059s
[4/1000] 1 tree, 31 leaves, max depth = 11, train score: 0.89040, val score: 0.85394, in 0.064s
[5/1000] 1 tree, 31 leaves, max depth = 11, train score: 0.89595, val score: 0.85530, in 0.066s
[6/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.90007, val score: 0.85618, in 0.065s
[7/1000] 1 tree, 31 leaves, max depth = 13, train score: 0.90363, val score: 0.85351, in 0.066s
[8/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.90634, val score: 0.86130, in 0.066s
[9/1000] 1 tree, 31 leaves, max depth = 11, train score: 0.91021, val score: 0.86198, in 0.071s
[10/1000] 1 tree,

(0.9658515572705477, 0.8836308129568515)

The algorithm may have stopped too early, so let's increase the threshold for early stopping by decreasing tol by a factor of 10 to e-8 from its default value e-7.

In [17]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1,
                                     max_iter=1000, scoring='roc_auc', tol=1e-8)
clf.fit(xs,y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

Binning 0.042 GB of training data: 0.271 s
Binning 0.005 GB of validation data: 0.019 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.78101, val score: 0.79729, in 0.058s
[2/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.81562, val score: 0.84685, in 0.083s
[3/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.84869, val score: 0.87510, in 0.047s
[4/1000] 1 tree, 31 leaves, max depth = 8, train score: 0.85485, val score: 0.88094, in 0.047s
[5/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.86568, val score: 0.88349, in 0.051s
[6/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.87742, val score: 0.88483, in 0.044s
[7/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.88160, val score: 0.88449, in 0.047s
[8/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.88430, val score: 0.88644, in 0.058s
[9/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.88651, val score: 0.88777, in 0.058s
[10/1000] 1 tree, 31 l

(0.9736014537452244, 0.8878831140620626)

In [41]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1, l2_regularization=2.4,
                             learning_rate=0.03, max_depth=25, max_iter=1000, max_leaf_nodes=44,
                             min_samples_leaf=8,  scoring='roc_auc', tol=1e-8)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

Binning 0.042 GB of training data: 0.277 s
Binning 0.005 GB of validation data: 0.020 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 44 leaves, max depth = 9, train score: 0.86889, val score: 0.82889, in 0.082s
[2/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.87925, val score: 0.82951, in 0.083s
[3/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.88058, val score: 0.83309, in 0.082s
[4/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.88378, val score: 0.83707, in 0.160s
[5/1000] 1 tree, 44 leaves, max depth = 9, train score: 0.88396, val score: 0.83800, in 0.085s
[6/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.88424, val score: 0.83771, in 0.088s
[7/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.88430, val score: 0.83762, in 0.087s
[8/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.88433, val score: 0.83813, in 0.087s
[9/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.88446, val score: 0.83836, in 0.090s
[10/1000] 1 tree,

[85/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.93967, val score: 0.88073, in 0.120s
[86/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.94014, val score: 0.88105, in 0.122s
[87/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.94048, val score: 0.88154, in 0.119s
[88/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.94068, val score: 0.88203, in 0.118s
[89/1000] 1 tree, 44 leaves, max depth = 16, train score: 0.94092, val score: 0.88201, in 0.119s
[90/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.94134, val score: 0.88236, in 0.202s
[91/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.94175, val score: 0.88235, in 0.111s
[92/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.94205, val score: 0.88261, in 0.117s
[93/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.94233, val score: 0.88285, in 0.116s
[94/1000] 1 tree, 44 leaves, max depth = 8, train score: 0.94257, val score: 0.88302, in 0.117s
[95/1000] 1 tree, 44 leaves, ma

[169/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.96550, val score: 0.89445, in 0.144s
[170/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.96576, val score: 0.89463, in 0.141s
[171/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.96603, val score: 0.89479, in 0.138s
[172/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.96607, val score: 0.89486, in 0.135s
[173/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.96627, val score: 0.89500, in 0.135s
[174/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.96658, val score: 0.89515, in 0.137s
[175/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.96689, val score: 0.89502, in 0.138s
[176/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.96698, val score: 0.89483, in 0.135s
[177/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.96725, val score: 0.89498, in 0.138s
[178/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.96737, val score: 0.89498, in 0.225s
[179/1000] 1 tree, 4

CPU times: user 3min 36s, sys: 37.1 s, total: 4min 13s
Wall time: 36.4 s


(0.9697907275897766, 0.8931305761072392)

In [59]:
xs_imp = (path/'xs_imp.pkl').load()
valid_xs_imp = (path/'valid_xs_imp.pkl').load()

In [61]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1, l2_regularization=2.4,
                             learning_rate=0.03, max_depth=25, max_iter=1000, max_leaf_nodes=44,
                             min_samples_leaf=8,  scoring='roc_auc', tol=1e-8)
clf.fit(xs_imp, y)
m_roc(y, xs_imp), m_roc(valid_y, valid_xs_imp)

Binning 0.012 GB of training data: 0.131 s
Binning 0.001 GB of validation data: 0.002 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.85396, val score: 0.84462, in 0.053s
[2/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.86097, val score: 0.84685, in 0.045s
[3/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.86322, val score: 0.85069, in 0.047s
[4/1000] 1 tree, 44 leaves, max depth = 9, train score: 0.86766, val score: 0.85036, in 0.046s
[5/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.87195, val score: 0.85396, in 0.046s
[6/1000] 1 tree, 44 leaves, max depth = 9, train score: 0.87285, val score: 0.85608, in 0.046s
[7/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.87627, val score: 0.85698, in 0.041s
[8/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.87640, val score: 0.85648, in 0.055s
[9/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.87772, val score: 0.85734, in 0.047s
[10/1000] 1 tree,

[85/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.93785, val score: 0.88402, in 0.085s
[86/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.93820, val score: 0.88433, in 0.127s
[87/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.93852, val score: 0.88422, in 0.076s
[88/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.93910, val score: 0.88455, in 0.066s
[89/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.93956, val score: 0.88435, in 0.067s
[90/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.94011, val score: 0.88468, in 0.064s
[91/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.94027, val score: 0.88481, in 0.062s
[92/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.94070, val score: 0.88475, in 0.067s
[93/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.94124, val score: 0.88505, in 0.064s
[94/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.94184, val score: 0.88534, in 0.069s
[95/1000] 1 tree, 44 leaves, m

[169/1000] 1 tree, 44 leaves, max depth = 18, train score: 0.96672, val score: 0.89366, in 0.075s
[170/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.96680, val score: 0.89365, in 0.079s
[171/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.96691, val score: 0.89404, in 0.077s
[172/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.96715, val score: 0.89408, in 0.124s
[173/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.96729, val score: 0.89414, in 0.076s
[174/1000] 1 tree, 44 leaves, max depth = 18, train score: 0.96756, val score: 0.89401, in 0.112s
[175/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.96772, val score: 0.89404, in 0.076s
[176/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.96784, val score: 0.89407, in 0.075s
[177/1000] 1 tree, 44 leaves, max depth = 9, train score: 0.96808, val score: 0.89474, in 0.077s
[178/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.96830, val score: 0.89475, in 0.076s
[179/1000] 1 tree, 44

(0.9633199712695736, 0.8972229710101309)

### Hyperparameter tuning

Hyperparameters were tuned interatively using GridSearchCV. Only the final results are displayed in the notenbook.

To speed up the grid search let's randomly sample 10% of our data (which is already a simple random sample of 10% of the original dataset).

In [171]:
xs_imp.shape, y.shape, valid_xs_imp.shape, valid_y.shape

((48357, 35), (48357,), (10697, 35), (10697,))

In [75]:
parameters1 = {
 'max_depth': [15],
 'random_state': [33],
 }

In [93]:
parameters2 = {
 'max_depth': [15],
 'learning_rate': [0.1],
 'random_state': [33],
 }

In [132]:
parameters3 = {
 'max_depth': [15],
 'learning_rate': [0.1], 
 'l2_regularization': [4.0],
 'random_state': [33],
 }

In [149]:
parameters4 = {
 'max_depth': [15],
 'learning_rate': [0.1], 
 'l2_regularization': [4.0],
 'min_samples_leaf': [20],
 'random_state': [33],
 }

In [159]:
parameters5 = {
 'max_depth': [15],
 'learning_rate': [0.1], 
 'l2_regularization': [4.0],
 'min_samples_leaf': [20],
 'max_leaf_nodes': [31],
 'random_state': [33],
 }

In [175]:
parameters6 = {
 'max_depth': [15],
 'learning_rate': [0.1], 
 'l2_regularization': [4.0],
 'min_samples_leaf': [20],
 'max_leaf_nodes': [31],
 'tol': [1e-8]
 'random_state': [33],
 }

In [176]:
clf = GridSearchCV(HistGradientBoostingClassifier(loss='binary_crossentropy', max_iter=1000, 
                                                  scoring='roc_auc', tol=1e-8), parameters6, 
                   scoring=make_scorer(roc_auc_score, needs_proba=True), n_jobs=-1, cv=3,
                   verbose=3)

In [177]:
%%time
clf.fit(xs, y)
clf.best_estimator_, clf.best_score_, clf.best_params_

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  12 | elapsed:  2.6min remaining: 12.8min
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:  4.7min remaining:  3.4min
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  5.9min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  5.9min finished


CPU times: user 3min 45s, sys: 1min 27s, total: 5min 13s
Wall time: 6min 48s


(HistGradientBoostingClassifier(l2_regularization=4.0,
                                loss='binary_crossentropy', max_depth=15,
                                max_iter=1000, random_state=33,
                                scoring='roc_auc', tol=1e-08),
 0.8963061409318414,
 {'l2_regularization': 4.0,
  'learning_rate': 0.1,
  'max_depth': 15,
  'max_leaf_nodes': 31,
  'min_samples_leaf': 20,
  'random_state': 33,
  'tol': 1e-08})

In [162]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1,
                                    max_iter=1000, scoring='roc_auc',
                                    max_depth=15, learning_rate=0.1,
                                    l2_regularization=4.0)
clf.fit(xs,y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

Binning 0.042 GB of training data: 0.274 s
Binning 0.005 GB of validation data: 0.020 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.83421, val score: 0.81920, in 0.052s
[2/1000] 1 tree, 31 leaves, max depth = 8, train score: 0.85031, val score: 0.83382, in 0.052s
[3/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.85215, val score: 0.83553, in 0.054s
[4/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.85447, val score: 0.84131, in 0.056s
[5/1000] 1 tree, 31 leaves, max depth = 11, train score: 0.85913, val score: 0.84563, in 0.062s
[6/1000] 1 tree, 31 leaves, max depth = 11, train score: 0.86568, val score: 0.85267, in 0.058s
[7/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.87155, val score: 0.85306, in 0.060s
[8/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.87469, val score: 0.85497, in 0.056s
[9/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.87696, val score: 0.85635, in 0.062s
[10/1000] 1 tree, 31

[85/1000] 1 tree, 31 leaves, max depth = 11, train score: 0.96630, val score: 0.90800, in 0.096s
[86/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.96665, val score: 0.90784, in 0.137s
[87/1000] 1 tree, 31 leaves, max depth = 14, train score: 0.96708, val score: 0.90771, in 0.119s
[88/1000] 1 tree, 31 leaves, max depth = 11, train score: 0.96727, val score: 0.90800, in 0.103s
[89/1000] 1 tree, 31 leaves, max depth = 13, train score: 0.96753, val score: 0.90812, in 0.110s
[90/1000] 1 tree, 31 leaves, max depth = 14, train score: 0.96787, val score: 0.90798, in 0.112s
[91/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.96831, val score: 0.90804, in 0.094s
[92/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.96881, val score: 0.90795, in 0.091s
[93/1000] 1 tree, 31 leaves, max depth = 14, train score: 0.96944, val score: 0.90793, in 0.092s
[94/1000] 1 tree, 31 leaves, max depth = 12, train score: 0.96968, val score: 0.90788, in 0.097s
[95/1000] 1 tree, 31 leaves, m

(0.9714914882971906, 0.8971570195158847)

### Including the V*** columns

In [194]:
to = (path/'to_full.pkl').load()

xs, y             = to.train.xs, to.train.y
valid_xs, valid_y = to.valid.xs, to.valid.y

In [195]:
cats = ['ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 
       'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 
       'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'DeviceType', 'DeviceInfo', 'id_12', 
       'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 
       'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 
       'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36',
       'id_37', 'id_38']

for cat in cats:
    to[cat] = to[cat].astype('category')

In [196]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1,
                                    max_iter=1000, scoring='roc_auc',
                                    max_depth=15, learning_rate=0.1,
                                    l2_regularization=4.0, tol=1e-8)
clf.fit(xs,y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

Binning 0.279 GB of training data: 1.745 s
Binning 0.031 GB of validation data: 0.042 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 31 leaves, max depth = 11, train score: 0.84101, val score: 0.85701, in 0.156s
[2/1000] 1 tree, 31 leaves, max depth = 8, train score: 0.86369, val score: 0.87599, in 0.177s
[3/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.86724, val score: 0.87988, in 0.157s
[4/1000] 1 tree, 31 leaves, max depth = 11, train score: 0.87065, val score: 0.88631, in 0.183s
[5/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.87563, val score: 0.89064, in 0.162s
[6/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.87965, val score: 0.89242, in 0.225s
[7/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.88089, val score: 0.89431, in 0.253s
[8/1000] 1 tree, 31 leaves, max depth = 11, train score: 0.88305, val score: 0.89610, in 0.255s
[9/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.88634, val score: 0.89780, in 0.227s
[10/1000] 1 tree, 31

[85/1000] 1 tree, 31 leaves, max depth = 11, train score: 0.95748, val score: 0.92662, in 0.304s
[86/1000] 1 tree, 31 leaves, max depth = 13, train score: 0.95765, val score: 0.92687, in 0.312s
[87/1000] 1 tree, 31 leaves, max depth = 15, train score: 0.95855, val score: 0.92688, in 0.317s
[88/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.95911, val score: 0.92699, in 0.492s
[89/1000] 1 tree, 31 leaves, max depth = 15, train score: 0.95927, val score: 0.92696, in 0.580s
[90/1000] 1 tree, 31 leaves, max depth = 12, train score: 0.95956, val score: 0.92701, in 0.340s
[91/1000] 1 tree, 31 leaves, max depth = 12, train score: 0.96012, val score: 0.92770, in 0.353s
[92/1000] 1 tree, 31 leaves, max depth = 14, train score: 0.96034, val score: 0.92785, in 0.309s
[93/1000] 1 tree, 31 leaves, max depth = 11, train score: 0.96072, val score: 0.92811, in 0.507s
[94/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.96092, val score: 0.92817, in 0.518s
[95/1000] 1 tree, 31 leaves, max

(0.9662431693097167, 0.8943595769682725)

### Only features deemed important by RF model interpretation with V*** features

In [205]:
xs_imp = (path/'xs_imp_full.pkl').load()
valid_xs_imp = (path/'valid_xs_imp_full.pkl').load()

In [206]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1,
                                    max_iter=1000, scoring='roc_auc',
                                    max_depth=15, learning_rate=0.1,
                                    l2_regularization=4.0, tol=1e-8)
clf.fit(xs_imp,y)
m_roc(y, xs_imp), m_roc(valid_y, valid_xs_imp)

Binning 0.011 GB of training data: 0.114 s
Binning 0.001 GB of validation data: 0.006 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.85183, val score: 0.82808, in 0.034s
[2/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.85846, val score: 0.83773, in 0.034s
[3/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.86037, val score: 0.84088, in 0.033s
[4/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.86785, val score: 0.85493, in 0.035s
[5/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.86885, val score: 0.85801, in 0.052s
[6/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.87304, val score: 0.85900, in 0.032s
[7/1000] 1 tree, 31 leaves, max depth = 11, train score: 0.87515, val score: 0.86176, in 0.032s
[8/1000] 1 tree, 31 leaves, max depth = 11, train score: 0.87517, val score: 0.86165, in 0.025s
[9/1000] 1 tree, 31 leaves, max depth = 8, train score: 0.87960, val score: 0.86407, in 0.042s
[10/1000] 1 tree, 3

(0.9433326003871689, 0.8825521657999629)

### Only features deemed important by RF model interpretation

In [207]:
xs_imp = (path/'xs_imp.pkl').load()
valid_xs_imp = (path/'valid_xs_imp.pkl').load()

In [208]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1,
                                    max_iter=1000, scoring='roc_auc',
                                    max_depth=15, learning_rate=0.1,
                                    l2_regularization=4.0, tol=1e-8)
clf.fit(xs_imp,y)
m_roc(y, xs_imp), m_roc(valid_y, valid_xs_imp)

Binning 0.012 GB of training data: 0.121 s
Binning 0.001 GB of validation data: 0.006 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.82711, val score: 0.81972, in 0.034s
[2/1000] 1 tree, 31 leaves, max depth = 8, train score: 0.84659, val score: 0.84106, in 0.041s
[3/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.85296, val score: 0.84471, in 0.039s
[4/1000] 1 tree, 31 leaves, max depth = 8, train score: 0.85699, val score: 0.84721, in 0.039s
[5/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.85718, val score: 0.84812, in 0.042s
[6/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.86644, val score: 0.85338, in 0.039s
[7/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.87393, val score: 0.85923, in 0.047s
[8/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.87638, val score: 0.86396, in 0.043s
[9/1000] 1 tree, 31 leaves, max depth = 8, train score: 0.87857, val score: 0.86645, in 0.046s
[10/1000] 1 tree, 31 l

CPU times: user 33.3 s, sys: 3.08 s, total: 36.4 s
Wall time: 5.53 s


(0.9554132480716929, 0.8944939662234133)